# AGDC-V2 API Functional Testing


##  Setting up environment: python modules etc

* Optional! 
* Skip this section if you use other means to ensure correct env setup, 
* such as module load before starting jupyter notebook.

In [1]:
import os, sys

# where is your lastest developer branch code?
my_agdcv2_src='/g/data1/u46/fxz547/Githubz/agdc-v2' 

sys.path.insert(0, my_agdcv2_src)  #prepend a path, to use the lastest developer branch

In [2]:
import imp; imp.find_module('datacube/api')  # check which python module file is being used

(None, '/g/data1/u46/fxz547/Githubz/agdc-v2/datacube/api', ('', '', 5))

In [3]:
from IPython.display import display
from pprint import pprint
from __future__ import print_function

from collections import defaultdict
import xarray as xr
import xarray.ufuncs

from datacube.api import API
from datacube.index import index_connect
from datacube.config import LocalConfig
from datacube.api._conversion import to_datetime
#new updated version: from datacube.api import make_mask, describe_flags

import numpy as np
import math

from pyproj import Proj, transform
from osgeo import gdal, osr, ogr
from rasterio.warp import reproject, RESAMPLING
import rasterio

%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10 # increase plot size a bit...
rcParams['axes.formatter.useoffset'] = False  # disable scalar formatter / offset in axes labels


Python envionment and modules are ready to go

## API with your specific configuration

In [4]:
force_prod = True
if force_prod:
    prod_config = LocalConfig.find(['/g/data/v10/public/modules/agdc-py2-prod/1.0.2/datacube.conf'])
    prod_index = index_connect(prod_config, application_name='api-WOfS-dev')
    dc = API(prod_index)
else:
    dc = API(application_name='api-WOfS-dev')

## List_cells ()

In [5]:
dc.list_cells(product='nbar', longitude=(149.06,149.18), latitude=(-35.27, -35.33) , time=('1996-01-01', '2016-03-20'))
#result =[(15, -40)]

[(15, -40)]

In [6]:
qdict={"product":"nbar", "platform": 'LANDSAT_5',"longitude": (149.06,149.18), "latitude":(-35.27, -35.33), "time":('1996-01-01', '2016-03-20')}

result=dc.list_cells(**qdict) #should use ** to unpack the dic into a kwarg

print (result)

print("But this return wrong results [] quietly")
result2 = dc.list_cells(qdict)

print (result2)

[(15, -40)]
But this return wrong results [] quietly
[]


In [7]:
# And this dict (for analytical engine style) also works
an_query = {
    'product': 'nbar',
    'platform': 'LANDSAT_5',
    'dimensions': {
        'x' : { 'range': (149.06, 149.18) },
        'y' : { 'range': (-35.27, -35.33) },
    'time': { 'range': ((1996, 1, 1), (2006, 7 ,1)) }
    }
}
result3=dc.list_cells(**an_query) # []
print (result3)

[(15, -40)]


## List Tiles

In [11]:
#define a cell covering Canberra
cells= [(15, -40)]

qdict={"product":"nbar", "platform": 'LANDSAT_5',"longitude": (149.06,149.18), "latitude":(-35.27, -35.33), "time":('1996-01-01', '2016-03-20')}

tiles = dc.list_tiles(cells, **qdict)

print(len(tiles))

print("Alternatively")
tiles = dc.list_tiles(cells,  product='nbar', platform='LANDSAT_5')#, time=('2000', '2005'))

print (len(tiles))


88
Alternatively
88


In [12]:
# But this call also returns results - appears incorrect 
tiles = dc.list_tiles(cells, qdict)

print(len(tiles))

659


In [10]:
nbar_tiles = dc.list_tiles(cells, product='nbar', platform='LANDSAT_8')


print (type(nbar_tiles))

print (len(nbar_tiles))
#print  nbar_tiles

pq_tiles = dc.list_tiles(cells, product='pqa', platform='LANDSAT_8') #, time=('2000', '2007'))
len(pq_tiles)
#print pq_tiles  # a huge list

<type 'list'>
68


68

## list field values are slow

In [ ]:
dc.list_field_values('product')

In [ ]:
dc.list_field_values('platform')

In [ ]:
# return a huge incomprehensible list. bug?
dc.list_all_field_values()  
# {'collection': [u'eo'],
#  u'gsi': [u'EDC', u'ASA', u'LGN', None],
#  'id': ['7fc31b32-e3f9-11e5-83a5-a0000100fe80',
#   'c092c80a-ec5b-11e5-80c7-a0000100fe80',
#   '776201cc-02b4-11e6-b767-a0000100fe80',
#   '9de6172e-e1d1-11e5-8b70-a0000100fe80',
#   '669fc2e4-e684-11e5-af28-a0000100fe80',